In [1]:
import jax.numpy as jnp
from jax import random, grad, jit, jacfwd, jacrev
from jax.scipy.linalg import inv, svd, eigh, det

from jax.lax import scan
import jax
from scipy.linalg import solve_discrete_are
from tqdm import tqdm
import matplotlib.pyplot as plt
from jax.tree_util import Partial

from jax_vi import KL_gaussian, log_likelihood, KL_sum
from jax_filters import apply_filtering_fixed_nonlinear, kalman_filter_process, filter_step
from jax_models import visualize_observations, Lorenz96, generate_true_states, generate_gc_localization_matrix




key = random.PRNGKey(3)

# System dimensions
n = 40  # System dimension
p = 2  # Observation dimension
J0 = 0 # burn in period
N = 10 # Monte Carlo samples
F = 8.0
dt = 0.05
num_steps = 1000  # Number of time steps

# Model parameters
m0 = jnp.ones((n,))
C0 = jnp.eye(n) * 1.0   # Initial state covariance matrix (P)
q = random.normal(key, (n, n))/5
Q = q@q.T + jnp.eye(n)*0.1    # Process noise covariance matrix (Sigma in Julia code)
Q = jnp.eye(n)*0.1   #jnp.eye(n) * 5.0    # Process noise covariance matrix (Sigma in Julia code)

H = jnp.eye(n)          # Observation matrix
# H = jnp.eye(n)[::2] #partial observation

R = jnp.eye(H.shape[0])  # R now becomes 20x20 for partial H 20*40
inv_R = inv(R)

observation_interval = 1

# State initialization
initial_state = random.normal(random.PRNGKey(0), (n,))  # Initial state

l96_model = Lorenz96(dt = dt, F = F)
state_transition_function = l96_model.step
l96_step = Partial(state_transition_function)
# Generate true states and observations using the Lorenz '96 model
key = random.PRNGKey(0)
jacobian_function = jacrev(l96_step, argnums=0)
jac_func = Partial(jacobian_function)


In [2]:
observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, l96_step, observation_interval)
y = observations

In [3]:
from functools import partial


@partial(jit, static_argnums=(3,10))
def var_cost(K, m0, C0, n, H, Q, R, y, key, num_steps, J0):
    states, covariances = apply_filtering_fixed_nonlinear(m0, C0, y, K, n, l96_step, jac_func, H, Q, R)
    key, *subkeys = random.split(key, num=N+1)
    kl_sum = KL_sum(states, covariances, n, l96_step, Q, key, N)

    def inner_map(subkey):
        return log_likelihood(random.multivariate_normal(subkey, states, covariances), y, H, R, num_steps, J0) 
    cost = kl_sum - jnp.nanmean(jax.lax.map(inner_map, jnp.vstack(subkeys)))
    print(cost)
    return cost


from functools import partial


@partial(jit, static_argnums=(3))
def var_cost_single_step(K, m0, C0, n, Q, H, R, y_curr, key, J, J0):
    (m_update, C_update), _  =  filter_step((m0,C0), y_curr, K, n, l96_step, jac_func, H, Q, R)
    
    log_likelihood_val = log_likelihood(m_update[jnp.newaxis, :], y_curr[jnp.newaxis, :], H, R, J=1, J0=J0)
    # Calculate the KL divergence between the predicted and updated state distributions
    m_pred = state_transition_function(m0)
    M = jac_func(m0)
    C_pred = M @ C0 @ M.T + Q
    kl_divergence = KL_gaussian(n, m_update, C_update, m_pred, C_pred)
    
    # Combine the KL divergence and the negative log-likelihood to form the cost
    cost = kl_divergence - log_likelihood_val
    return cost


In [4]:
m, C, K = kalman_filter_process(l96_step, jac_func, m0, C0, observations, H, Q, R)
K_steady = jnp.mean(K[-10:, :, :], axis=0)
K_steady

Array([[ 2.9307029e-01, -1.0042849e-02, -2.6016304e-02, ...,
        -1.1984844e-02,  1.9772753e-02,  4.0613603e-02],
       [-1.0042855e-02,  2.8541455e-01, -6.8869330e-02, ...,
        -1.4934078e-02,  2.1845214e-02,  6.7896418e-02],
       [-2.6016304e-02, -6.8869330e-02,  2.7601305e-01, ...,
        -2.3392761e-04,  3.4112309e-03, -7.6177530e-03],
       ...,
       [-1.1984842e-02, -1.4934080e-02, -2.3392645e-04, ...,
         2.7485043e-01, -2.1949125e-02, -5.0493028e-02],
       [ 1.9772749e-02,  2.1845208e-02,  3.4112323e-03, ...,
        -2.1949127e-02,  2.7540705e-01,  3.0512437e-02],
       [ 4.0613603e-02,  6.7896418e-02, -7.6177469e-03, ...,
        -5.0493028e-02,  3.0512441e-02,  2.5728223e-01]], dtype=float32)

In [6]:
### K_opt for partial observation!

if H.shape[0] == 40:
    K_opt = jnp.eye(n) * 0.4
else:
    K_opt = jnp.zeros((40, 20))
    for i in range(0, K_opt.shape[1]):
        K_opt = K_opt.at[i*2, i].set(1)
    #K_opt = K_opt + random.normal(key, K_opt.shape) * 0.6

In [7]:
var_cost(K_opt, m0, C0, n, H, Q, R, y, key, num_steps, J0)

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


Array(154061.31, dtype=float32)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from jax.numpy import linalg as jnpl
from tqdm.auto import tqdm

var_cost_grad = grad(var_cost, argnums=0)

# Initial guess for K and optimization parameters

alpha = 1e-8

live = False
prediction_errors = [] 
norms = []
true_div = []

n_iters = 500
num_steps = n_iters

for i in tqdm(range(n_iters)):
    key, _ = random.split(key)
    # Update the gradient and Kalman gain
    grad_K = var_cost_grad(K_opt, m0, C0, n, H, Q, R, y, key, num_steps, J0)
    K_opt -= alpha * grad_K
    
    # Apply filtering with the newly optimized K to generate state predictions
    predicted_states, covariances = apply_filtering_fixed_nonlinear(m0, C0, y, K_opt, n, l96_step, jac_func, H, Q, R)
    
    prediction_error = np.mean(np.mean((predicted_states - true_states)**2, axis=1))
    prediction_errors.append(prediction_error)
    norms.append(jnpl.norm(K_opt - K_steady))
    total_kl_divergence = 0
    for t in range(num_steps):  
        kl_div_t = KL_gaussian(n, predicted_states[t], covariances[t],  m[t], C[t])
        total_kl_divergence += kl_div_t
    
    true_div.append(total_kl_divergence / num_steps)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from jax.numpy import linalg as jnpl
# from tqdm.auto import tqdm


# true_div = []
# prediction_errors = [] 
# norms = []
# Ks = []
# live = True

# # Define the gradient of the cost function
# var_cost_single_grad = grad(var_cost_single_step, argnums = 0)

# # Initial guess for K and optimization parameters
# alpha = 1e-5

# num_steps = 1000
# for i in tqdm(range(num_steps)):
#     key, _ = random.split(key)
#     y_curr = observations[i] 
#     # Update the gradient and Kalman gain
#     for j in range(100):
#         grad_K = var_cost_single_grad(K_opt, m0, C0, n, Q, H, R, y_curr, key, num_steps, J0)
#         K_opt -= alpha * grad_K
#     Ks.append(K_opt)
#     norms.append(jnp.linalg.norm(K_opt - K_steady)) 
#     (m_update, C_update), _ = filter_step((m0,C0), y_curr, K_opt, n, l96_step, jac_func, H, Q, R)
#     prediction_error = np.square(m_update - true_states[i]).mean()  # Assuming true_states[i] is available
#     prediction_errors.append(prediction_error)
#     true_div.append(KL_gaussian(n, m_update, C_update, m[i], C[i]))
#     # Prepare for the next step
#     m0, C0 = m_update, C_update



In [ ]:
# fig, ax1 = plt.subplots(figsize=(7, 4))

# color = 'tab:red'
# ax1.set_xlabel('Iteration')
# ax1.set_ylabel('$\|K_\mathrm{opt} - K_\mathrm{steady}\|_F$', color=color)
# ax1.plot(norms, label='Gain error', color=color)
# ax1.tick_params(axis='y', labelcolor=color)

# # Instantiate a second y-axis that shares the same x-axis
# ax2 = ax1.twinx()
# color = 'tab:blue'
# ax2.set_ylabel('Prediction error (MSE)', color=color)
# ax2.plot(prediction_errors, label='Prediction error (MSE)', color=color, linestyle='--')
# ax2.tick_params(axis='y', labelcolor=color)

# # Title and legend
# fig.tight_layout()
# #plt.title('Optimization and Prediction Errors over Iterations')
# fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)


# if live:
#     plt.savefig("nonlinear_gain_errors_live.pdf")
# else:
#     plt.savefig("nonlinear_gain_errors.pdf")
# #plt.show()

In [ ]:
import os
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)
ax1.pcolormesh(K_steady, vmin=-0.1, vmax=0.45, cmap='RdBu_r')
ax1.set_title('$K_\mathrm{steady}$')
p2 = ax2.pcolormesh(K_opt, vmin=-0.1, vmax=0.45, cmap='RdBu_r')
ax2.set_title('$K_\mathrm{opt}$')
cb_ax = fig.add_axes([.93,.124,.02,.754])
fig.colorbar(p2,orientation='vertical',cax=cb_ax)

subfolder_name = 'nonlinear_results'
os.makedirs(subfolder_name, exist_ok=True)

file_base_name = "nonlinear_gain_matrices"
if H.shape[0] == 20:
    file_base_name += "_partial"
if live:
    file_base_name += "_live"
file_name = file_base_name + ".pdf"

file_path = os.path.join(subfolder_name, file_name)
plt.savefig(file_path)
    

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax3) = plt.subplots(figsize=(10, 4), ncols=2)

# Optimization Error
color = 'tab:red'
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Gain error ($\|K_\mathrm{opt} - K_\mathrm{steady}\|_F$)', color=color)
line1, = ax1.plot(range(1, n_iters+1), norms[:n_iters], label='Gain error', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Instantiate a second y-axis for Prediction Error and True Divergence
ax2 = ax1.twinx()
color_pred = 'tab:green'
ax2.set_ylabel('KL divergence to true filter', color=color_pred)
#line1, = ax2.plot(prediction_errors, label='Prediction Error (MSE)', color=color_pred, linestyle='--')
line2, = ax2.plot(range(1, n_iters+1), true_div[:n_iters], label='KL divergence to true filter', color=color_pred, linestyle='-.')
ax2.tick_params(axis='y', labelcolor=color_pred)

# Title and combined legend
#plt.title('Gain errors and KL divergence over iterations')

# Creating a combined legend for all lines
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

ax3.plot(range(1, n_iters+1), prediction_errors[:n_iters])
ax3.set_xlabel("Iteration")
ax3.set_ylabel("Prediction error (MSE)")

plt.tight_layout()
#plt.show()
subfolder_name = 'nonlinear_results'
os.makedirs(subfolder_name, exist_ok=True)

file_base_name = "nonlinear_gain"
if H.shape[0] == 20:
    file_base_name += "_partial"
if live:
    file_base_name += "_live"
file_name = file_base_name + ".pdf"

file_path = os.path.join(subfolder_name, file_name)
plt.savefig(file_path)